# 30-Day Trend Analysis

This notebook generates rolling 30-day trend tables with DuckDB pivots and HTML export.


In [1]:
import sys
sys.path.append('../')

import polars as pl
import duckdb
import json
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np

# Set up paths
DATA_DIR = Path("../data")
LABELS_FILE = DATA_DIR / "labels_initial.parquet"
REGISTRY_FILE = Path("../registry/topic_registry.json")
OUTPUT_DIR = Path("../output")
OUTPUT_DIR.mkdir(exist_ok=True)

print("✓ Setup complete")


✓ Setup complete


In [2]:

# Load data
labels_df = pl.read_parquet(LABELS_FILE)
if labels_df.is_empty():
    raise ValueError(f"No topic assignments found in {LABELS_FILE}")
print(f"✓ Loaded {len(labels_df):,} topic assignments")

required_columns = {'topic_id', 'created_at'}
missing = required_columns - set(labels_df.columns)
if missing:
    raise ValueError(f"labels parquet missing required columns: {missing}")

# Load registry for topic names
with open(REGISTRY_FILE) as f:
    registry = json.load(f)

registry_version = registry.get('version', 'unknown')
topic_map = {t['id']: t['name'] for t in registry['topics']}
print(f"✓ Loaded {len(topic_map)} topic definitions (registry v{registry_version})")


✓ Loaded 27,381 topic assignments
✓ Loaded 32 topic definitions (registry v001)


## Generate 30-Day Pivot Table


In [3]:

# Calculate date range (last 31 days)
end_date = datetime.now().date()
start_date = end_date - timedelta(days=30)

print(f"📅 Date range: {start_date} to {end_date}")

# Convert created_at to date
labels_dated = labels_df.with_columns([
    pl.col("created_at").dt.date().alias("date")
])

# Filter to date range
labels_filtered = labels_dated.filter(
    (pl.col("date") >= start_date) & (pl.col("date") <= end_date)
)

if labels_filtered.is_empty():
    raise ValueError('No topic assignments found in the last 30 days. Check routing outputs or adjust the window.')

print(f"✓ Filtered to {len(labels_filtered):,} assignments in date range")
print(f"  Unique topics in window: {labels_filtered['topic_id'].n_unique()}")
print(f"  Unique dates in window: {labels_filtered['date'].n_unique()}")


📅 Date range: 2025-09-28 to 2025-10-28
✓ Filtered to 27,381 assignments in date range
  Unique topics in window: 33
  Unique dates in window: 28


In [4]:
# Use DuckDB for efficient pivot
conn = duckdb.connect()

# Create temp view
conn.register('labels', labels_filtered)

# Get unique dates
dates = labels_filtered["date"].unique().sort().to_list()
if not dates:
    raise ValueError('No dates available after filtering. Ensure labels contain recent data.')

date_strs = [d.strftime("%Y-%m-%d") for d in dates]

# Generate pivot query
sql = """
SELECT 
    topic_id,
"""
for date_str in date_strs:
    sql += f'''    SUM(CASE WHEN date = DATE '{date_str}' THEN 1 ELSE 0 END) AS "{date_str}",\n'''

sql = sql.rstrip(",\n") + """
FROM labels
WHERE date IS NOT NULL
GROUP BY topic_id
ORDER BY topic_id
"""

trend_df = conn.execute(sql).df()
if trend_df.empty:
    raise ValueError('Pivot table returned no data. Verify topic assignments for the selected window.')

print(f"✓ Created pivot table: {len(trend_df)} topics × {len(date_strs)} dates")


✓ Created pivot table: 33 topics × 28 dates


## Compute 7-Day Change Metrics


In [5]:

# Calculate 7-day change percentage for each topic
trend_pd = trend_df.set_index('topic_id')

if len(date_strs) >= 14:
    recent_window = date_strs[-7:]
    prev_window = date_strs[-14:-7]
    recent_7_days = trend_pd[recent_window].sum(axis=1)
    prev_7_days = trend_pd[prev_window].sum(axis=1)

    with np.errstate(divide='ignore', invalid='ignore'):
        change_pct = ((recent_7_days - prev_7_days) / prev_7_days.replace(0, np.nan) * 100)
    change_pct = change_pct.fillna(0).astype(float)

    trend_df['7d_change_pct'] = change_pct.values
    trend_df['recent_7d_total'] = recent_7_days.values
    trend_df['prev_7d_total'] = prev_7_days.values

    print(f"✓ Calculated 7-day change percentages across {len(recent_window)} recent days")
else:
    trend_df['7d_change_pct'] = 0.0
    trend_df['recent_7d_total'] = 0
    trend_df['prev_7d_total'] = 0
    print('⚠ Not enough dates for 7-day change calculation; filled with zeros')


✓ Calculated 7-day change percentages across 7 recent days


## Generate Sparklines (Unicode)


In [6]:

# Generate sparklines using Unicode block characters
def generate_sparkline(values):
    """Generate Unicode sparkline from list of values"""
    if not values or all(v == 0 for v in values):
        return "▁▁▁▁▁▁▁"

    min_val, max_val = min(values), max(values)
    if max_val == min_val:
        return "▃▃▃▃▃▃▃"

    normalized = [(v - min_val) / (max_val - min_val) * 7 for v in values]
    blocks = "▁▂▃▄▅▆▇█"

    sparkline = "".join([blocks[int(min(7, round(v)))] for v in normalized])
    return sparkline

# Add sparklines
sparklines = []
for _, row in trend_pd.iterrows():
    values = [row[date_str] for date_str in date_strs]
    sparklines.append(generate_sparkline(values))

trend_df['sparkline'] = sparklines
trend_df['latest_date'] = date_strs[-1]
trend_df['latest_count'] = trend_df[date_strs[-1]]
print("✓ Generated sparklines")


✓ Generated sparklines


## Export to CSV and HTML


In [7]:
# Save CSV
csv_file = OUTPUT_DIR / f"topics_trend_{end_date}.csv"
trend_df.to_csv(csv_file, index=False)
print(f"✓ Saved CSV to {csv_file}")

# Save enriched CSV for debugging
csv_debug_file = OUTPUT_DIR / f"topics_trend_{end_date}_debug.csv"
debug_columns = ['topic_id', 'latest_count', 'recent_7d_total', 'prev_7d_total', '7d_change_pct', 'sparkline'] + date_strs
debug_df = trend_df[debug_columns]
debug_df.to_csv(csv_debug_file, index=False)
print(f"  Debug CSV saved to {csv_debug_file}")

# Generate HTML
html = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Topic Trends - {end_date}</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 20px; }}
        table {{ border-collapse: collapse; width: 100%; }}
        th, td {{ padding: 8px; text-align: left; border-bottom: 1px solid #ddd; }}
        th {{ background-color: #f2f2f2; }}
        .positive {{ color: green; }}
        .negative {{ color: red; }}
    </style>
</head>
<body>
    <h1>Topic Trends: Last 30 Days</h1>
    <p>Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
    <table>
        <tr>
            <th>Topic ID</th>
            <th>Topic Name</th>
            <th>Trend</th>
            <th>7d Δ%</th>
            <th>Recent Total</th>
            <th>Prev Total</th>
"""

for date_str in date_strs[-7:]:  # Show last 7 dates
    html += f"            <th>{date_str}</th>\n"
html += "        </tr>\n"

for _, row in trend_df.iterrows():
    topic_id = row['topic_id']
    topic_name = topic_map.get(topic_id, topic_id)
    sparkline = row['sparkline']
    change_pct = row['7d_change_pct']

    change_class = 'positive' if change_pct > 0 else ('negative' if change_pct < 0 else '')
    change_str = f"{change_pct:+.1f}%"

    html += f"        <tr>\n"
    html += f"            <td>{topic_id}</td>\n"
    html += f"            <td>{topic_name}</td>\n"
    html += f"            <td style='font-family: monospace;'>{sparkline}</td>\n"
    html += f"            <td class='{change_class}'>{change_str}</td>\n"
    html += f"            <td>{int(row['recent_7d_total'])}</td>\n"
    html += f"            <td>{int(row['prev_7d_total'])}</td>\n"

    for date_str in date_strs[-7:]:
        html += f"            <td>{int(row[date_str])}</td>\n"

    html += "        </tr>\n"

html += """
    </table>
</body>
</html>
"""

html_file = OUTPUT_DIR / f"topics_trend_{end_date}.html"
html_file.write_text(html)
print(f"✓ Saved HTML to {html_file}")

# Verify artifacts exist
for artifact in [csv_file, html_file, csv_debug_file]:
    if not artifact.exists():
        raise FileNotFoundError(f"Missing expected artifact: {artifact}")

print("\n📊 Top 10 Topics (by recent activity):")
latest_col = date_strs[-1]
top_topics = trend_df.nlargest(10, latest_col)
for _, row in top_topics.iterrows():
    topic_id = row['topic_id']
    topic_name = topic_map.get(topic_id, topic_id)
    count = row[latest_col]
    change = row['7d_change_pct']
    print(f"  {topic_name}: {count} reviews ({change:+.1f}%)")


✓ Saved CSV to ../output/topics_trend_2025-10-28.csv
  Debug CSV saved to ../output/topics_trend_2025-10-28_debug.csv
✓ Saved HTML to ../output/topics_trend_2025-10-28.html

📊 Top 10 Topics (by recent activity):
  Positive Experience: 228.0 reviews (-5.4%)
  NOVEL: 87.0 reviews (+5.0%)
  Negative Generic: 86.0 reviews (-6.2%)
  Very Good Service: 52.0 reviews (-7.3%)
  Late Delivery: 43.0 reviews (-10.2%)
  Good Quality Food: 37.0 reviews (-3.8%)
  Great App: 35.0 reviews (-9.5%)
  Unprofessional Behavior: 33.0 reviews (-14.1%)
  High Fees: 29.0 reviews (-18.5%)
  Bot Only No Human Support: 27.0 reviews (-12.7%)
